In [1]:
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from collections import Counter


In [7]:
unembedding_matrix = torch.load('./save_common_pt/unembedding_matrix_7B.pt')
last_token_before_wo_wait = torch.load('tmp3.pt')


/tmp/ipykernel_18385/896269943.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unembedding_matrix = torch.load('./save_common_pt/unembedding_matrix_7B.pt')
/tmp/ipykerne

In [12]:
print(len(last_token_before_wo_wait))

1024


In [19]:
rank_list = []
layers = 29
numerical_ranks = []
numerical_ranks_no_wait = []
for layer in tqdm(range(layers)):
    token_before_wait = torch.cat([
        x[layer][-1] if x[layer].shape[1] > 1 else x[layer]
        for x in last_token_before_wo_wait
    ]).squeeze().to("cuda:0")
    words = []
    for token in token_before_wait:
        logits = torch.matmul(token, unembedding_matrix.T)
        words.append(logits.topk(1).indices.item())
    
    # token_before_no_wait = torch.cat([
    #     x[layer][-1] if x[layer].shape[1] > 1 else x[layer]
    #     for x in last_token_before_no_wait
    # ]).squeeze().to("cuda:0").to(torch.float32)
    # Apply logit lens for the current layer
    # logits = torch.matmul(token_before_wait, unembedding_matrix.T)
    # log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
    # rank = torch.argsort(log_probs, dim=-1, descending=True)
    counter = Counter(words)

    top3 = counter.most_common(3)
    rank_list.append(top3)


100%|██████████| 29/29 [00:36<00:00,  1.27s/it]


In [28]:
print(rank_list)

[[(105059, 44), (75258, 42), (51679, 37)], [(4028, 771), (133129, 67), (68106, 34)], [(4028, 883), (133129, 67), (68106, 17)], [(4028, 925), (133129, 65), (68106, 3)], [(4028, 952), (133129, 50), (127457, 2)], [(4028, 949), (133129, 43), (68106, 3)], [(4028, 994), (133129, 16), (134572, 1)], [(4028, 986), (133129, 20), (71720, 2)], [(4028, 931), (133129, 33), (68106, 5)], [(4028, 928), (133129, 36), (87965, 8)], [(4028, 949), (133129, 30), (71720, 5)], [(4028, 944), (133129, 35), (71720, 6)], [(4028, 905), (133129, 46), (71720, 9)], [(4028, 899), (133129, 45), (142157, 10)], [(4028, 905), (133129, 47), (71720, 14)], [(4028, 953), (71720, 19), (133129, 18)], [(4028, 932), (133129, 20), (71720, 18)], [(4028, 963), (133129, 18), (71720, 16)], [(4028, 964), (133129, 18), (71720, 17)], [(4028, 961), (71720, 19), (133129, 17)], [(4028, 975), (133129, 24), (71720, 10)], [(4028, 985), (133129, 22), (71720, 6)], [(4028, 971), (133129, 35), (71720, 4)], [(4028, 955), (133129, 52), (141351, 3)], 

In [20]:
# Load the tokenizer (already imported from transformers)
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
char_list = []
for layer, tokens in enumerate(rank_list):
    char_list.append([])
    for token in tokens:
        char_list[layer].append(tokenizer.decode(token[0]))
print(char_list)

[['以下简称', '/\n\n\n', 'evity'], ['\ufeffusing', 'โปรแ', " './../../"], ['\ufeffusing', 'โปรแ', " './../../"], ['\ufeffusing', 'โปรแ', " './../../"], ['\ufeffusing', 'โปรแ', 'ロン'], ['\ufeffusing', 'โปรแ', " './../../"], ['\ufeffusing', 'โปรแ', 'ぜひ'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', 'โปรแ', " './../../"], ['\ufeffusing', 'โปรแ', '"crypto'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', 'โปรแ', ' Республик'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', '"sync', 'โปรแ'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', '"sync', 'โปรแ'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', 'โปรแ', ' Diễn'], ['\ufeffusing', 'โปรแ', ' sure'], ['\ufeffusing', 'โปรแ', ' proportional'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ', 'تكو'], [',', ' ', ' the']]


In [21]:
for idx, char_ in enumerate(char_list):
    print(f"layer{idx}: {char_}")

layer0: ['以下简称', '/\n\n\n', 'evity']
layer1: ['\ufeffusing', 'โปรแ', " './../../"]
layer2: ['\ufeffusing', 'โปรแ', " './../../"]
layer3: ['\ufeffusing', 'โปรแ', " './../../"]
layer4: ['\ufeffusing', 'โปรแ', 'ロン']
layer5: ['\ufeffusing', 'โปรแ', " './../../"]
layer6: ['\ufeffusing', 'โปรแ', 'ぜひ']
layer7: ['\ufeffusing', 'โปรแ', '"sync']
layer8: ['\ufeffusing', 'โปรแ', " './../../"]
layer9: ['\ufeffusing', 'โปรแ', '"crypto']
layer10: ['\ufeffusing', 'โปรแ', '"sync']
layer11: ['\ufeffusing', 'โปรแ', '"sync']
layer12: ['\ufeffusing', 'โปรแ', '"sync']
layer13: ['\ufeffusing', 'โปรแ', ' Республик']
layer14: ['\ufeffusing', 'โปรแ', '"sync']
layer15: ['\ufeffusing', '"sync', 'โปรแ']
layer16: ['\ufeffusing', 'โปรแ', '"sync']
layer17: ['\ufeffusing', 'โปรแ', '"sync']
layer18: ['\ufeffusing', 'โปรแ', '"sync']
layer19: ['\ufeffusing', '"sync', 'โปรแ']
layer20: ['\ufeffusing', 'โปรแ', '"sync']
layer21: ['\ufeffusing', 'โปรแ', '"sync']
layer22: ['\ufeffusing', 'โปรแ', '"sync']
layer23: ['\ufeffusing

In [29]:
tokenizer.decode(4028)

'\ufeffusing'

In [ ]:
vocab = tokenizer.get_vocab()

id_to_token = {v: k for k, v in vocab.items()}

id_to_token[4028]

'ï»¿using'

In [ ]:
import torch
import torch.nn.functional as F


# 计算某一行（假设是 unembedding matrix 第 100 行）
index = 4028
vector = unembedding_matrix[index]

# Select the tokenizer weight

W_E = unembedding_matrix.detach()  # Detach the unembedding matrix to avoid gradients
similarities = F.cosine_similarity(vector.unsqueeze(0), W_E, dim=1)
most_similar_index = torch.argmax(similarities).item()

# 查找 token
token = tokenizer.convert_ids_to_tokens(most_similar_index)
print(f"Unembedding 矩阵第 {index} 行最接近的 token: {token}")


Unembedding 矩阵第 4028 行最接近的 token: ï»¿using


In [32]:
print(len(vocab))

151665


In [37]:
print(id_to_token[151663])

<|repo_name|>


In [14]:
pos_char_list = [['swer', 'AVA'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ', ' Diễn'], ['\ufeffusing', 'โปรแ', ' Diễn'], ['\ufeffusing'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ', ' similarly'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['Wait', 'But', 'So']]

for idx, char_ in enumerate(pos_char_list):
    print(f"layer{idx}: {char_}")

layer0: ['swer', 'AVA']
layer1: ['\ufeffusing']
layer2: ['\ufeffusing']
layer3: ['\ufeffusing']
layer4: ['\ufeffusing']
layer5: ['\ufeffusing', 'โปรแ']
layer6: ['\ufeffusing']
layer7: ['\ufeffusing']
layer8: ['\ufeffusing']
layer9: ['\ufeffusing']
layer10: ['\ufeffusing']
layer11: ['\ufeffusing', 'โปรแ']
layer12: ['\ufeffusing', 'โปรแ']
layer13: ['\ufeffusing', 'โปรแ', ' Diễn']
layer14: ['\ufeffusing', 'โปรแ', ' Diễn']
layer15: ['\ufeffusing']
layer16: ['\ufeffusing', 'โปรแ']
layer17: ['\ufeffusing']
layer18: ['\ufeffusing']
layer19: ['\ufeffusing', 'โปรแ', '"sync']
layer20: ['\ufeffusing', 'โปรแ']
layer21: ['\ufeffusing', 'โปรแ']
layer22: ['\ufeffusing', 'โปรแ', ' similarly']
layer23: ['\ufeffusing', 'โปรแ']
layer24: ['\ufeffusing', 'โปรแ']
layer25: ['\ufeffusing', 'โปรแ']
layer26: ['\ufeffusing', 'โปรแ']
layer27: ['\ufeffusing', 'โปรแ']
layer28: ['Wait', 'But', 'So']


In [61]:
last_token_before_wait = torch.load('last_token_before_wait3.pt')
# last_token_before_no_wait = torch.load('last_token_before_wo_wait.pt')
unembedding_matrix = torch.load('./save_common_pt/unembedding_matrix_7B.pt')
# last_token_before_wo_wait = torch.load('last_token_before_wo_wait2.pt')

/tmp/ipykernel_256502/248964222.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  last_token_before_wait = torch.load('last_token_before_wait3.pt')
/tmp/ipykernel_256502/2

In [71]:
rank_list = []
layers = 29
numerical_ranks = []
numerical_ranks_no_wait = []
for layer in tqdm(range(layers)):
    layer = 28
    token_before_wait = torch.cat([
        x[layer][-1] if x[layer].shape[1] > 1 else x[layer]
        for x in last_token_before_wait
    ]).squeeze().to("cuda:0").to(torch.float32)
    # token_before_no_wait = torch.cat([
    #     x[layer][-1] if x[layer].shape[1] > 1 else x[layer]
    #     for x in last_token_before_no_wait
    # ]).squeeze().to("cuda:0").to(torch.float32)
    # Apply logit lens for the current layer
    logits = torch.matmul(token_before_wait, unembedding_matrix.T.to(torch.float32))
    print(logits.topk(5))
    break
    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
    rank = torch.argsort(log_probs, dim=-1, descending=True)
    counter = Counter(rank[:, 0].cpu().numpy())

    top3 = counter.most_common(3)
    rank_list.append(top3)


  0%|          | 0/29 [00:00<?, ?it/s]

torch.return_types.topk(
values=tensor([[34.4442, 34.1548, 33.0454, 32.7614, 32.6894],
        [29.9377, 29.6828, 28.5877, 28.2374, 28.0513],
        [50.6681, 41.6015, 40.5407, 39.7558, 38.5367],
        [44.8483, 42.3410, 38.3209, 37.3414, 35.6974],
        [43.5858, 38.0402, 37.7300, 36.7073, 36.5469],
        [39.8190, 38.3588, 37.9600, 37.8474, 37.3483],
        [40.4755, 37.3365, 36.3555, 36.1515, 35.8439],
        [40.7275, 38.2709, 38.2261, 38.1205, 37.3120],
        [39.1513, 38.3095, 38.0626, 38.0134, 36.6637],
        [38.8602, 37.5165, 36.7772, 35.2375, 34.6005],
        [41.4242, 40.1922, 38.5785, 37.9638, 37.2456],
        [39.2162, 38.0098, 37.4870, 37.1154, 36.1609],
        [40.6199, 40.4620, 37.2815, 36.8132, 36.5902],
        [37.6144, 36.0664, 34.6528, 34.2430, 34.0148],
        [42.0404, 38.9795, 38.7331, 38.2692, 38.0011],
        [39.2819, 37.3451, 36.9219, 35.9209, 34.2213],
        [36.4837, 34.7594, 34.5188, 34.1464, 33.0332],
        [38.2882, 37.7499, 37.137

In [63]:
print(rank_list)

[]


In [6]:
# Load the tokenizer (already imported from transformers)
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
char_list = []
for layer, tokens in enumerate(rank_list):
    char_list.append([])
    for token in tokens:
        char_list[layer].append(tokenizer.decode(token[0]))
print(char_list)

[['swer', 'AVA'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ', ' Diễn'], ['\ufeffusing', 'โปรแ', ' Diễn'], ['\ufeffusing'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing', 'โปรแ', '"sync'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ', ' similarly'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['\ufeffusing', 'โปรแ'], ['Wait', 'But', 'So']]


In [ ]:
print(wait_list)

[11489, 14190, 3783, 13824]


In [ ]:
tokenizer.decode([rank_list[0][0][0], rank_list[1][0][0], rank_list[2][0][0]])

'.\n\n.\n\n.\n\n'

In [ ]:
tokenizer.additional_special_tokens

[]

In [1]:
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from collections import Counter


In [41]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
# TODO: why wait and space wait is different, why .\n\n is a token instead of .

model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map="auto", torch_dtype=torch.bfloat16
)
# last_token_before_wait = torch.load('last_token_before_wait3.pt')

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [44]:
import torch
import torch.nn.functional as F


# 计算某一行（假设是 unembedding matrix 第 100 行）
index = 14190
vector = unembedding_matrix[index]

# Select the tokenizer weight
embedding_matrix = model.get_input_embeddings().weight
W_E = embedding_matrix.detach()  # Detach the embedding matrix to avoid gradients
similarities = F.cosine_similarity(vector.unsqueeze(0), W_E, dim=0)
most_similar_index = torch.argmax(similarities).item()

# 查找 token
token = tokenizer.convert_ids_to_tokens(most_similar_index)
print(f"Unembedding 矩阵第 {index} 行最接近的 token: {token}")


Unembedding 矩阵第 14190 行最接近的 token: _st


In [4]:
unembedding_matrix = torch.load('./save_common_pt/unembedding_matrix_7B.pt')
norm_layer = torch.load('norm_layer.pt')

/tmp/ipykernel_274095/984210714.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unembedding_matrix = torch.load('./save_common_pt/unembedding_matrix_7B.pt')
/tmp/ipykern

In [4]:
rotary_emb_layer = torch.load('rotary_emb_layer.pt')

/tmp/ipykernel_256502/1164291981.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rotary_emb_layer = torch.load('rotary_emb_layer.pt')


In [5]:
lm_head = torch.load('lm_head.pt')
# model.lm_head.load_state_dict(lm_head)

/tmp/ipykernel_256502/3378495784.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lm_head = torch.load('lm_head.pt')


In [45]:
print(len(last_token_before_wait))

19


In [52]:
rank_list = []
layers = 29
numerical_ranks = []
numerical_ranks_no_wait = []
for layer in range(layers):
    token_before_wait = torch.cat([
        x[layer][-1] if x[layer].shape[1] > 1 else x[layer]
        for x in last_token_before_wait
    ]).squeeze().to("cuda:0")
    # token_before_wait = token_before_wait * norm_layer['weight']
    # token_before_wait = token_before_wait * rotary_emb_layer.get('weight', torch.ones_like(token_before_wait))
    logits = torch.matmul(token_before_wait, unembedding_matrix.T)
    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
    rank = torch.argsort(log_probs, dim=-1, descending=True)
    counter = Counter(rank[:, 0].cpu().numpy())

    top3 = counter.most_common(3)
    rank_list.append(top3)


In [ ]:
[11489, 14190, 3783, 13824]


In [53]:
# Load the tokenizer (already imported from transformers)
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
char_list = []
for layer, tokens in enumerate(rank_list):
    char_list.append([])
    for token in tokens:
        char_list[layer].append(tokenizer.decode(token[0]))
print(char_list)

[[' Laura'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], ['\ufeffusing'], [',']]


In [50]:
print(rank_list)

[[(11, 19)]]


In [7]:
selected_layers = [model.model.norm, model.model.rotary_emb]
torch.save(selected_layers[0].state_dict(), 'norm_layer.pt')
torch.save(selected_layers[1].state_dict(), 'rotary_emb_layer.pt')

In [27]:
print(model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,), eps=1e-06)
    (rotary_emb):

In [3]:
embedding_matrix = torch.load('./save_common_pt/embedding_matrix_7B.pt')
unembedding_matrix = torch.load('./save_common_pt/unembedding_matrix_7B.pt')

are_identical = torch.allclose(embedding_matrix, unembedding_matrix, atol=1e-6)
print("Are embedding and unembedding matrices identical?", are_identical)


/tmp/ipykernel_274541/2066913298.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embedding_matrix = torch.load('./save_common_pt/embedding_matrix_7B.pt')
/tmp/ipykernel_

Are embedding and unembedding matrices identical? False


In [ ]:
# Assuming the model is loaded as `model`
lm_head = model.lm_head
print(lm_head)

torch.save(lm_head.state_dict(), 'lm_head.pt')

Linear(in_features=3584, out_features=152064, bias=False)


In [ ]:
unembedding_matrix = model.lm_head.weight
print(unembedding_matrix)